<h1><center>Simple YoloX Dataset Generator (COCO-JSON)</center></h1>     

<center><img src = "https://i.imgur.com/iatgdo5.jpg" width = "635" height = "235"/></center>         

This dataset was built to be compatible with the train (train.py) script that can be found [HERE](https://github.com/Megvii-BaseDetection/YOLOX). I also have a training notebook that you can find [HERE](https://www.kaggle.com/coldfir3/yolox). The inference notebook is still WIP. The resulting kaggle Dataset cand be found [HERE](https://www.kaggle.com/coldfir3/cots-yolox-dataset)

The tree main tasks into converting this dataset to YoloX format are:
1. Splitting into train and test
1. Converting the bboxes to COCO format
1. Saving the annotations to a .json file

I took inspiration on [this](https://www.kaggle.com/remekkinas/yolox-training-pipeline-cots-dataset-lb-0-507) amazing notebook

<h3 style='background:orange; color:black'><center>Consider upvoting this notebook if you found it helpful.</center></h3>

## Loading the DataFrame and spliting into train/test

I have a whole discussion topic ([here](https://www.kaggle.com/c/tensorflow-great-barrier-reef/discussion/293723)) where I explain why I think spliting by video is the best approach, but feel free to change the train/val split strategy

In [ ]:
from ast import literal_eval
import pandas as pd
from tqdm.notebook import tqdm

TRAIN_PATH = '/kaggle/input/tensorflow-great-barrier-reef/train_images'
N_SAMP = 6000

df = pd.read_csv('/kaggle/input/tensorflow-great-barrier-reef/train.csv')
n_with_annotations = (df['annotations'] != '[]').sum()

df = pd.concat([
    df[df['annotations'] != '[]'],
    df[df['annotations'] == '[]'].sample(N_SAMP - n_with_annotations)
]).sample(frac=1).reset_index(drop = True)

df['is_valid'] = df['video_id'] == 2
df['annotations'] = df['annotations'].apply(literal_eval)
df['path'] = df.apply(lambda row: f"{TRAIN_PATH}/video_{row['video_id']}/{row['video_frame']}.jpg", axis = 1)

df.tail()

## Converting to COCO

In [ ]:
def coco(df):
    
    annotion_id = 0
    images = []
    annotations = []

    categories = [{'id': 0, 'name': 'cots'}]

    for i, row in tqdm(df.iterrows(), total = len(df)):

        images.append({
            "id": i,
            "file_name": f"{row['image_id']}.jpg",
            "height": 720,
            "width": 1280,
        })
        for bbox in row['annotations']:
            annotations.append({
                "id": annotion_id,
                "image_id": i,
                "category_id": 0,
                "bbox": list(bbox.values()),
                "area": bbox['width'] * bbox['height'],
                "segmentation": [],
                "iscrowd": 0
            })
            annotion_id += 1

    json_file = {'categories':categories, 'images':images, 'annotations':annotations}
    return json_file

In [ ]:
json_train = coco(df[~df['is_valid']])
json_valid = coco(df[ df['is_valid']])

## Saving into a json file

In [ ]:
import json

with open('/kaggle/working/annotations_train.json', 'w', encoding='utf-8') as f:
    json.dump(json_train, f, ensure_ascii=True, indent=4)
    
with open('/kaggle/working/annotations_valid.json', 'w', encoding='utf-8') as f:
    json.dump(json_valid, f, ensure_ascii=True, indent=4)

In [ ]:
import os
os.makedirs('/kaggle/working/train2017', exist_ok=True)
os.makedirs('/kaggle/working/val2017', exist_ok=True)

In [ ]:
import shutil
for i, row in tqdm(df.iterrows(), total = len(df)):
    base_dir = 'val2017' if row['is_valid'] else 'train2017'
    fname = f"{row['image_id']}.jpg"
    shutil.copyfile(row['path'], f"/kaggle/working/{base_dir}/{fname}")

## Sanity check

In [ ]:
!pip install -Uqqq 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
from PIL import Image
from random import sample

In [ ]:
data_dir = '/kaggle/working/train2017'
ann_file = '/kaggle/working/annotations_train.json'
coco = COCO(ann_file)
img_ids = coco.getImgIds()

In [ ]:
n_row = 3
n_col = 2
imgs = coco.loadImgs(sample(img_ids, n_row * n_col))
_, axs = plt.subplots(n_row, n_col, figsize=(12 * n_col, 8 * n_row))
axs = axs.flatten()
for img, ax in zip(imgs, axs):
    img_img = Image.open(f"{data_dir}/{img['file_name']}")
    anns = coco.loadAnns(coco.getAnnIds(imgIds=[img['id']]))
    ax.imshow(img_img)
    plt.sca(ax)
    coco.showAnns(anns, draw_bbox=True)
    plt.axis('off')
plt.show()

## Ziping the files so kaggle can assemble a dataset

the final dataset can be found [HERE](https://www.kaggle.com/coldfir3/great-barrier-reef-yolov5)

In [ ]:
shutil.make_archive('val2017', 'zip', 'val2017')
shutil.make_archive('train2017', 'zip', 'train2017')

In [ ]:
shutil.rmtree('val2017') 
shutil.rmtree('train2017') 